# 3D mesh to model the full FTES experiment

This script generates the mesh used in section 3.3.6 (Modelling of FTES experiment) of my master's thesis report.

## Generate 2D geometry of domain cross-section

In [1]:
import gmsh
import numpy as np
from math import cos, sin, pi

# initialize the model
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)

L = 250e-3                     # size of the square
r_hole = 17e-3/2               # radius of the hole
r_tubing_outer = 3.175e-3/2    # outer radius of the tubing
r_tubing_inner = 1e-3/2        # inner radius of the tubing

# Define the points to create the quarter circle
geom = gmsh.model.geo
pc = geom.addPoint(0, 0, 0) 
p1 = geom.addPoint(r_tubing_inner, 0, 0) 
p2 = geom.addPoint(r_tubing_outer, 0, 0) 
p3 = geom.addPoint(r_hole, 0, 0) 
p4 = geom.addPoint(L/2, 0, 0) 
p5 = geom.addPoint(L/2, L/2, 0) 
p6 = geom.addPoint(0, L/2, 0) 
p7 = geom.addPoint(0, r_hole, 0)
p8 = geom.addPoint(0, r_tubing_outer, 0)
p9 = geom.addPoint(0, r_tubing_inner, 0)
p10 = geom.addPoint(r_tubing_inner*cos(pi/4), r_tubing_inner*sin(pi/4), 0)
p11 = geom.addPoint(r_tubing_outer*cos(pi/4), r_tubing_outer*sin(pi/4), 0)
p12 = geom.addPoint(r_hole*cos(pi/4), r_hole*sin(pi/4), 0)

# Define lines from points
l1 = geom.addLine(pc, p1)
l2 = geom.addLine(p1, p2)
l3 = geom.addLine(p2, p3)
l4 = geom.addLine(p3, p4)
l5 = geom.addLine(p4, p5)
l6 = geom.addLine(p5, p6)
l7 = geom.addLine(p6, p7)
l8 = geom.addLine(p7, p8)
l9 = geom.addLine(p8, p9)
l10 = geom.addLine(p9, pc)
l11 = geom.addLine(pc, p10)
l12 = geom.addLine(p10, p11)
l13 = geom.addLine(p11, p12)
l14 = geom.addLine(p12, p5)
c1 = geom.addCircleArc(p1, pc, p10)
c2 = geom.addCircleArc(p10, pc, p9)
c3 = geom.addCircleArc(p2, pc, p11)
c4 = geom.addCircleArc(p11, pc, p8)
c5 = geom.addCircleArc(p3, pc, p12)
c6 = geom.addCircleArc(p12, pc, p7)

# Create curve loops
cl1 = geom.addCurveLoop([l1, c1, c2, l10])
cl2 = geom.addCurveLoop([l2, c3, -l12, -c1])
cl3 = geom.addCurveLoop([l12, c4, l9, -c2])
cl4 = geom.addCurveLoop([l3, c5, -l13, -c3])
cl5 = geom.addCurveLoop([l13, c6, l8, -c4])
cl6 = geom.addCurveLoop([l5, -l14, -c5, l4]) 
cl7 = geom.addCurveLoop([l6, l7, -c6, l14])

# Define transfinite curves as we want a structured mesh
for tag in [c1, c2, c3, c4, c5, c6, l5, l6, l1, l10]:
    geom.mesh.setTransfiniteCurve(tag, 20)
for tag in [l4, l14, l7]:
    geom.mesh.setTransfiniteCurve(tag, 20)
for tag in [l3, l13, l8]:
    geom.mesh.setTransfiniteCurve(tag, 8)
for tag in [l2, l12, l9]:
    geom.mesh.setTransfiniteCurve(tag, 8)
    
# Define surfaces
s_circle_tubing_inner = geom.addPlaneSurface([cl1])
s_circle_tubing_outer_down = geom.addPlaneSurface([cl2])
s_circle_tubing_outer_up = geom.addPlaneSurface([cl3])
s_circle_hole_down = geom.addPlaneSurface([cl4])
s_circle_hole_up = geom.addPlaneSurface([cl5])
s_down = geom.addPlaneSurface([cl6])
s_up = geom.addPlaneSurface([cl7])

# Define transfinite surfaces to get a structured mesh
surface_list = [s_circle_tubing_inner, s_circle_tubing_outer_down, s_circle_tubing_outer_up, s_circle_hole_down, s_circle_hole_up, s_down, s_up]
for surf in surface_list:
    geom.mesh.setTransfiniteSurface(surf)

# Recombine to use quadrilateral elements instead of triangles
for surf in surface_list:
    geom.mesh.setRecombine(2, surf)

## Extrude by steps

In [2]:
# First extrusion (height of L/2)
bottom_top_surf = []
bottom_vol = []
for surf in surface_list:
    bottom = gmsh.model.geo.extrude([(2, surf)], 0, 0, L/2, [1]*20, list(np.around(np.linspace(0, 1, num=21)**(1/2), decimals=3)[1:]), recombine=True)
    bottom_top_surf.append(bottom[0])
    bottom_vol.append(bottom[1][1])

In [3]:
# Second extrusion (height of 500 microns)
fracture_width = 0.001/2
middle_top_surf = []
middle_vol = []
for i, surf in enumerate(bottom_top_surf):
    middle = gmsh.model.geo.extrude([surf], 0, 0, fracture_width, [20], recombine=True)  #10
    middle_top_surf.append(middle[0])
    middle_vol.append(middle[1][1])
    if i==5:
        right_frac_face = middle[2][1]
    elif i==6:
        up_frac_face = middle[2][1]

In [4]:
# Third extrusion (height of L/2)
top_top_surf = []
top_vol = []
for surf in middle_top_surf:
    top = gmsh.model.geo.extrude([surf], 0, 0, L/2, [50], recombine=True)
    top_top_surf.append(top[0])
    top_vol.append(top[1][1])

The interior of the tubing is extruded with an additional height of L/300 to force the temperature gradient to be perpendicular to the Dirichlet boundary condition and thus facilitate the calculation of the energy given by conduction to the system.

In [5]:
# Fourth extrusion only for the inner circle (height of L/300)
top_extension_top_surf = []
top_extension_vol = []
top_extension = gmsh.model.geo.extrude([top_top_surf[0]], 0, 0, L/300, [20], recombine=True)
top_extension_top_surf = top_extension[0]
top_extension_vol = top_extension[1][1]

In [6]:
geom.synchronize()

## Define volumes for solid materials

In [7]:
# Metal domain
gmsh.model.addPhysicalGroup(3, top_vol[1:3], 103)
gmsh.model.setPhysicalName(3, 103, 'metal')

# Epoxy domain
gmsh.model.addPhysicalGroup(3, bottom_vol[:5] + top_vol[3:5], 104)
gmsh.model.setPhysicalName(3, 104, 'epoxy')

# Rock domain
gmsh.model.addPhysicalGroup(3, bottom_vol[5:] + top_vol[5:], 105)
gmsh.model.setPhysicalName(3, 105, 'rock')

## Define volumes for fluid flow

In [8]:
# Fluid domain 1 = inside the upper part of the inner tubing
gmsh.model.addPhysicalGroup(3, [top_vol[0], middle_vol[0], top_extension_vol], 106)
gmsh.model.setPhysicalName(3, 106, 'well')

# Fluid domain 2 = inside the fracture
gmsh.model.addPhysicalGroup(3, middle_vol[1:], 107)
gmsh.model.setPhysicalName(3, 107, 'fracture')

## Define surfaces for boundary conditions and energy calculations

In [9]:
# Natural boundary condition (0 flux) on all surfaces except at the well's 
# inlet, where the water temperature is prescribed (Dirichlet boundary 
# condition).
gmsh.model.addPhysicalGroup(2, [top_extension_top_surf[1]], 1)
gmsh.model.setPhysicalName(2, 1, 'inletTemperature')

gmsh.model.addPhysicalGroup(2, [right_frac_face, up_frac_face], 2)
gmsh.model.setPhysicalName(2, 2, 'outletTemperature')

## Generate mesh

In [10]:
# Generate mesh
gmsh.model.mesh.generate(3)

# Save mesh
filename = '3D_FTES_mesh.msh'
gmsh.write(filename)

gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Circle)
Info    : [ 20%] Meshing curve 16 (Circle)
Info    : [ 20%] Meshing curve 17 (Circle)
Info    : [ 20%] Meshing curve 18 (Circle)
Info    : [ 20%] Meshing curve 19 (Circle)
Info    : [ 20%] Meshing curve 20 (Circle)
Info    : [ 20%] Meshing curve 22 (Extruded)
Info    : [ 20%] Meshing curve 23 (Extruded)
Info    : [ 20%] Meshing curve 24 (Extruded)
Info    : [ 20%] M